In [128]:
%pip install geopy 
%pip install scikit-learn
%pip install pandas
%pip install xgboost


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [130]:
import pandas as pd
import numpy as np
import xgboost as xgb

import pickle
from geopy.geocoders import Nominatim
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

# Import data and take a look at it

In [131]:
df = pd.read_csv("DetailLichTrinh.csv")
df

,ID,Name,Address,Rating,Rating_Count,Open_Close,Description,Latitude,Longitude,Sessional,TypeTravel,TourID,Ordinal_Numbers
0,1,Nhà thờ Domaine de Marie,"1 Ngô Quyền, Phường 6, Thành phố Đà Lạt, Lâm Đ...",4.5,6084,07:00–17:30,"Nhà thờ màu hồng nổi tiếng, tọa lạc trong khun...",11.949671,108.430247,Sáng,Tham quan,1,1
1,2,DINH BẢO ĐẠI III,"1 Đường Triệu Việt Vương, Phường 4, Thành phố ...",4.1,7232,07:00–17:30,Cung điện rộng lớn của vị hoàng đế cuối cùng t...,11.930313,108.429577,Sáng,Tham quan,1,2
2,3,Ga Đà Lạt,"7a Nguyễn Trãi, Phường 10, Thành phố Đà Lạt, L...",4.3,2254,Cả ngày,"Đầu tiên, bạn gửi xe tại bãi xe của ga Đà Lạt....",11.941541,108.454256,Chiều,Tham quan,1,3
3,4,Chùa ve chai,"Trại Mát, Thành phố Đà Lạt, Lâm Đồng, Việt Nam",4.6,8899,07:00–17:30,"Ngôi chùa lớn và ấn tượng, nổi tiếng với những...",11.944670,108.499416,Chiều,Tham quan,1,4
4,5,Chợ Đêm Đà Lạt,"6b Đường Nguyễn Thị Minh Khai, Phường 1, Thành...",4.2,11479,Cả ngày,"Khu chợ tấp nập các quầy hàng bán đồ gia dụng,...",11.941584,108.437279,Tối,Ăn uống,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,91,Đồi Chè Cầu Đất,"VHG6+VFX, Phát Chi, Thành phố Đà Lạt, Lâm Đồng...",4.4,2192,Cả ngày,Đồi chè rất đẹp. Rộng rãi. Bây giờ có thêm mấy...,11.877419,108.561183,Chiều,Tham quan,9,4
90,92,Thanh Nhu,"XC8V+GGC, Phường 8, Thành phố Đà Lạt, Lâm Đồng...",3.8,38,07:00–22:00,"Quán có không gian rộng rãi, bày bán giá cả cô...",11.966490,108.443794,Chiều,Tham quan,9,5
91,93,Buffet rau - Thế giới rau ngon,"17 Đ. Nguyễn Chí Thanh, Phường 1, Thành phố Đà...",3.2,185,07:00–17:30,"Quán rộng rãi, nhân viên cực dễ thương nhiệt t...",11.942312,108.436875,Tối,Ăn uống,9,6
92,94,Chợ Đêm Đà Lạt,"6b Đường Nguyễn Thị Minh Khai, Phường 1, Thành...",4.2,11479,Cả ngày,"Khu chợ tấp nập các quầy hàng bán đồ gia dụng,...",11.941584,108.437279,Sáng,Ăn uống,9,1


In [132]:
df['ID'] = df['ID'].fillna(0).astype(int)
df['TourID'] = df['TourID'].fillna(0).astype(int)
df['Ordinal_Numbers'] = df['Ordinal_Numbers'].fillna(0).astype(int)

In [133]:
df.shape

(94, 13)

In [134]:
df.head()

,ID,Name,Address,Rating,Rating_Count,Open_Close,Description,Latitude,Longitude,Sessional,TypeTravel,TourID,Ordinal_Numbers
0,1,Nhà thờ Domaine de Marie,"1 Ngô Quyền, Phường 6, Thành phố Đà Lạt, Lâm Đ...",4.5,6084,07:00–17:30,"Nhà thờ màu hồng nổi tiếng, tọa lạc trong khun...",11.949671,108.430247,Sáng,Tham quan,1,1
1,2,DINH BẢO ĐẠI III,"1 Đường Triệu Việt Vương, Phường 4, Thành phố ...",4.1,7232,07:00–17:30,Cung điện rộng lớn của vị hoàng đế cuối cùng t...,11.930313,108.429577,Sáng,Tham quan,1,2
2,3,Ga Đà Lạt,"7a Nguyễn Trãi, Phường 10, Thành phố Đà Lạt, L...",4.3,2254,Cả ngày,"Đầu tiên, bạn gửi xe tại bãi xe của ga Đà Lạt....",11.941541,108.454256,Chiều,Tham quan,1,3
3,4,Chùa ve chai,"Trại Mát, Thành phố Đà Lạt, Lâm Đồng, Việt Nam",4.6,8899,07:00–17:30,"Ngôi chùa lớn và ấn tượng, nổi tiếng với những...",11.944670,108.499416,Chiều,Tham quan,1,4
4,5,Chợ Đêm Đà Lạt,"6b Đường Nguyễn Thị Minh Khai, Phường 1, Thành...",4.2,11479,Cả ngày,"Khu chợ tấp nập các quầy hàng bán đồ gia dụng,...",11.941584,108.437279,Tối,Ăn uống,1,5


Giải thích ý nghĩa của các fields

id: Đây có thể là ID duy nhất của mỗi chuyến đi, không trùng lặp.
vendor_id: Định danh nhà cung cấp dịch vụ.
pickup_datetime: Thời điểm bắt đầu chuyến đi.
dropoff_datetime: Thời điểm kết thúc chuyến đi.
passenger_count: Số lượng hành khách trong chuyến đi.
pickup_longitude: Kinh độ nơi lấy hành khách.
pickup_latitude: Vĩ độ nơi lấy hành khách.
dropoff_longitude: Kinh độ nơi trả hành khách.
dropoff_latitude: Vĩ độ nơi trả hành khách.
store_and_fwd_flag: Có lưu trữ và chuyển tiếp dữ liệu không.
trip_duration: Thời lượng chuyến đi (có thể tính bằng đơn vị giây hoặc phút).
pickup_month: Tháng khi bắt đầu chuyến đi.
pickup_day: Ngày trong tháng khi bắt đầu chuyến đi.
pickup_weekday: Ngày trong tuần khi bắt đầu chuyến đi.
pickup_hour: Giờ khi bắt đầu chuyến đi.
pickup_minute: Phút khi bắt đầu chuyến đi.
latitude_difference: Độ chênh lệch vĩ độ giữa nơi lấy và nơi trả hành khách.
longitude_difference: Độ chênh lệch kinh độ giữa nơi lấy và nơi trả hành khách.
trip_distance: Khoảng cách của chuyến đi (có thể tính bằng đơn vị mét hoặc mile).

In [135]:
sample_df = df.drop_duplicates(subset='Name', keep='first')

In [136]:
#Reset ID
sample_df['ID'] = range(1, len(sample_df) + 1)

C:\Users\long.nguyen\AppData\Local\Temp\ipykernel_8768\1555318037.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['ID'] = range(1, len(sample_df) + 1)


In [137]:
len(sample_df)

52

In [138]:
sample_df.head()

,ID,Name,Address,Rating,Rating_Count,Open_Close,Description,Latitude,Longitude,Sessional,TypeTravel,TourID,Ordinal_Numbers
0,1,Nhà thờ Domaine de Marie,"1 Ngô Quyền, Phường 6, Thành phố Đà Lạt, Lâm Đ...",4.5,6084,07:00–17:30,"Nhà thờ màu hồng nổi tiếng, tọa lạc trong khun...",11.949671,108.430247,Sáng,Tham quan,1,1
1,2,DINH BẢO ĐẠI III,"1 Đường Triệu Việt Vương, Phường 4, Thành phố ...",4.1,7232,07:00–17:30,Cung điện rộng lớn của vị hoàng đế cuối cùng t...,11.930313,108.429577,Sáng,Tham quan,1,2
2,3,Ga Đà Lạt,"7a Nguyễn Trãi, Phường 10, Thành phố Đà Lạt, L...",4.3,2254,Cả ngày,"Đầu tiên, bạn gửi xe tại bãi xe của ga Đà Lạt....",11.941541,108.454256,Chiều,Tham quan,1,3
3,4,Chùa ve chai,"Trại Mát, Thành phố Đà Lạt, Lâm Đồng, Việt Nam",4.6,8899,07:00–17:30,"Ngôi chùa lớn và ấn tượng, nổi tiếng với những...",11.944670,108.499416,Chiều,Tham quan,1,4
4,5,Chợ Đêm Đà Lạt,"6b Đường Nguyễn Thị Minh Khai, Phường 1, Thành...",4.2,11479,Cả ngày,"Khu chợ tấp nập các quầy hàng bán đồ gia dụng,...",11.941584,108.437279,Tối,Ăn uống,1,5


## Engineer features

In [140]:
import numpy as np
from itertools import combinations

def calculate_distance(lat1, lon1, lat2, lon2):
    p = np.pi/180
    a = 0.5 - np.cos((lat2-lat1)*p)/2 + np.cos(lat1*p) * np.cos(lat2*p) * (1-np.cos((lon2-lon1)*p))/2
    return 12742 * np.arcsin(np.sqrt(a)) # 2*R*asin...

def create_pairs(sample_df):
    pairs = []
    for i, row1 in sample_df.iterrows():
        for j, row2 in sample_df.iterrows():
            if i < j:
                lat1, lon1 = row1['Latitude'], row1['Longitude']
                lat2, lon2 = row2['Latitude'], row2['Longitude']
                distance = calculate_distance(lat1, lon1, lat2, lon2)
                distance_id = f'point_{i+1}_{j+1}'
                pairs.append((distance_id, distance))
    return pairs
print(create_pairs(sample_df))
pairs = create_pairs(sample_df)

[('point_1_2', 2.153778095665176), ('point_1_3', 2.763894548784014), ('point_1_4', 7.545110156630424), ('point_1_5', 1.1806210504349581), ('point_1_6', 1.9725468797056824), ('point_1_7', 3.770053493398163), ('point_1_8', 2.4870506312350042), ('point_1_9', 4.099876372143504), ('point_1_10', 3.6861137507817854), ('point_1_11', 5.484544879730545), ('point_1_12', 5.574827138400698), ('point_1_13', 5.480750492331693), ('point_1_14', 23.325522291839775), ('point_1_15', 11.079131399503607), ('point_1_16', 17.435174724497816), ('point_1_20', 14.14705283189875), ('point_1_22', 4.968721524549582), ('point_1_23', 3.424532476537181), ('point_1_24', 9.015467803716113), ('point_1_25', 9.061207567530765), ('point_1_29', 4.72470146668276), ('point_1_30', 6.892815058022429), ('point_1_31', 3.285254653187611), ('point_1_36', 10.751756984231776), ('point_1_38', 33.985404667594366), ('point_1_39', 4.3300117933912174), ('point_1_42', 1.0906131937012666), ('point_1_46', 8.54508203256499), ('point_1_49', 2.1

In [141]:
len(pairs)

1326

In [142]:
pairs

[('point_1_2', 2.153778095665176),
 ('point_1_3', 2.763894548784014),
 ('point_1_4', 7.545110156630424),
 ('point_1_5', 1.1806210504349581),
 ('point_1_6', 1.9725468797056824),
 ('point_1_7', 3.770053493398163),
 ('point_1_8', 2.4870506312350042),
 ('point_1_9', 4.099876372143504),
 ('point_1_10', 3.6861137507817854),
 ('point_1_11', 5.484544879730545),
 ('point_1_12', 5.574827138400698),
 ('point_1_13', 5.480750492331693),
 ('point_1_14', 23.325522291839775),
 ('point_1_15', 11.079131399503607),
 ('point_1_16', 17.435174724497816),
 ('point_1_20', 14.14705283189875),
 ('point_1_22', 4.968721524549582),
 ('point_1_23', 3.424532476537181),
 ('point_1_24', 9.015467803716113),
 ('point_1_25', 9.061207567530765),
 ('point_1_29', 4.72470146668276),
 ('point_1_30', 6.892815058022429),
 ('point_1_31', 3.285254653187611),
 ('point_1_36', 10.751756984231776),
 ('point_1_38', 33.985404667594366),
 ('point_1_39', 4.3300117933912174),
 ('point_1_42', 1.0906131937012666),
 ('point_1_46', 8.54508203

In [143]:
#Combine pairs into a dataframe
pairs_df = pd.DataFrame(pairs, columns=['distance_id', 'distance'])
sample_df = sample_df.assign(Distance_ID=pairs_df['distance_id'], Distance=pairs_df['distance'])
sample_df.head()


,ID,Name,Address,Rating,Rating_Count,Open_Close,Description,Latitude,Longitude,Sessional,TypeTravel,TourID,Ordinal_Numbers,Distance_ID,Distance
0,1,Nhà thờ Domaine de Marie,"1 Ngô Quyền, Phường 6, Thành phố Đà Lạt, Lâm Đ...",4.5,6084,07:00–17:30,"Nhà thờ màu hồng nổi tiếng, tọa lạc trong khun...",11.949671,108.430247,Sáng,Tham quan,1,1,point_1_2,2.153778
1,2,DINH BẢO ĐẠI III,"1 Đường Triệu Việt Vương, Phường 4, Thành phố ...",4.1,7232,07:00–17:30,Cung điện rộng lớn của vị hoàng đế cuối cùng t...,11.930313,108.429577,Sáng,Tham quan,1,2,point_1_3,2.763895
2,3,Ga Đà Lạt,"7a Nguyễn Trãi, Phường 10, Thành phố Đà Lạt, L...",4.3,2254,Cả ngày,"Đầu tiên, bạn gửi xe tại bãi xe của ga Đà Lạt....",11.941541,108.454256,Chiều,Tham quan,1,3,point_1_4,7.545110
3,4,Chùa ve chai,"Trại Mát, Thành phố Đà Lạt, Lâm Đồng, Việt Nam",4.6,8899,07:00–17:30,"Ngôi chùa lớn và ấn tượng, nổi tiếng với những...",11.944670,108.499416,Chiều,Tham quan,1,4,point_1_5,1.180621
4,5,Chợ Đêm Đà Lạt,"6b Đường Nguyễn Thị Minh Khai, Phường 1, Thành...",4.2,11479,Cả ngày,"Khu chợ tấp nập các quầy hàng bán đồ gia dụng,...",11.941584,108.437279,Tối,Ăn uống,1,5,point_1_6,1.972547


In [144]:
#Add duration
sample_df['Trip_Duration'] = sample_df['Distance'] / 40 * 60
sample_df.head()

,ID,Name,Address,Rating,Rating_Count,Open_Close,Description,Latitude,Longitude,Sessional,TypeTravel,TourID,Ordinal_Numbers,Distance_ID,Distance,Trip_Duration
0,1,Nhà thờ Domaine de Marie,"1 Ngô Quyền, Phường 6, Thành phố Đà Lạt, Lâm Đ...",4.5,6084,07:00–17:30,"Nhà thờ màu hồng nổi tiếng, tọa lạc trong khun...",11.949671,108.430247,Sáng,Tham quan,1,1,point_1_2,2.153778,3.230667
1,2,DINH BẢO ĐẠI III,"1 Đường Triệu Việt Vương, Phường 4, Thành phố ...",4.1,7232,07:00–17:30,Cung điện rộng lớn của vị hoàng đế cuối cùng t...,11.930313,108.429577,Sáng,Tham quan,1,2,point_1_3,2.763895,4.145842
2,3,Ga Đà Lạt,"7a Nguyễn Trãi, Phường 10, Thành phố Đà Lạt, L...",4.3,2254,Cả ngày,"Đầu tiên, bạn gửi xe tại bãi xe của ga Đà Lạt....",11.941541,108.454256,Chiều,Tham quan,1,3,point_1_4,7.545110,11.317665
3,4,Chùa ve chai,"Trại Mát, Thành phố Đà Lạt, Lâm Đồng, Việt Nam",4.6,8899,07:00–17:30,"Ngôi chùa lớn và ấn tượng, nổi tiếng với những...",11.944670,108.499416,Chiều,Tham quan,1,4,point_1_5,1.180621,1.770932
4,5,Chợ Đêm Đà Lạt,"6b Đường Nguyễn Thị Minh Khai, Phường 1, Thành...",4.2,11479,Cả ngày,"Khu chợ tấp nập các quầy hàng bán đồ gia dụng,...",11.941584,108.437279,Tối,Ăn uống,1,5,point_1_6,1.972547,2.958820


# Modeling

In [145]:
#tiền Xử lý về dạng số và drop những loại chữ
# sessional_mapping = {'Sáng': 1, 'Chiều': 2, 'Tối': 3, 'Trưa': 4, 'Tất cả': 5}
# sample_df['Sessional'] = sample_df['Sessional'].map(sessional_mapping)
sample_df['Sessional'] = sample_df['Sessional'].astype('category')

# type_mapping = {'Vui chơi': 1, 'Tham quan': 2, 'Ăn uống': 3, 'Lịch sử': 4, 'Âm nhạc': 5}
# sample_df['TypeTravel'] = sample_df['TypeTravel'].map(type_mapping)
sample_df['TypeTravel'] = sample_df['TypeTravel'].astype('category')

# open_close_mapping = {'07:00-17:30': 1, '07:00-22:00': 2, 'Cả ngày': 3}
# sample_df['Open_Close'] = sample_df['Open_Close'].map(open_close_mapping)
sample_df['Open_Close'] = sample_df['Open_Close'].astype('category')

sample_df.head()

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:

X = sample_df.drop(["Trip_Duration", "ID", "Name", "Address", "Open_Close", "Distance_ID", "Description",], axis=1)
y = sample_df["Trip_Duration"]

In [116]:
#Split the data into training, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2022)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2023)

In [117]:
#Define evaluation metric
def rmsle(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    return np.square(np.log(y_pred + 1) - np.log(y_true + 1)).mean() ** 0.5

In [118]:
#XGBoost parameters 
params = {
    'booster':            'gbtree', #Loại mô hình cây tăng cường
    'objective':          'reg:linear', # Đối tượng huẩn luyện: Mô hình hồi quy tuyến tính
    'learning_rate':      0.05, # Tỷ lệ học của mô hình
    'max_depth':          14, # Độ sâu tối đa của cây
    'subsample':          0.9, # Tỷ lệ mẫu được sử dụng để huấn luyện cây
    'colsample_bytree':   0.7, # Tỷ lệ các cột được sử dụng để huấn luyện cây
    'colsample_bylevel':  0.7, # Tỷ lệ các cột được sử dụng để huấn luyện mỗi cây
    'silent':             1, # Không in quá trình huấn luyện
    'feval':              'rmsle' # Đánh giá mô hình
}

In [119]:
nrounds = 2000 # Số lượng vòng lặp

In [121]:
#Define train and validation sets
dtrain = xgb.DMatrix(X_train, np.log(y_train+1)) #np.log(y_train+1) is used to transform the target variable to log scale
dval = xgb.DMatrix(X_val, np.log(y_val+1)) #np.log(y_val+1) is used to transform the target variable to log scale

#this is for tracking the error
watchlist = [(dval, 'eval'), (dtrain, 'train')] 

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Name: object, Address: object, Open_Close: object, Sessional: object, TypeTravel: object

In [42]:
#Train model
gbm = xgb.train(params,
                dtrain,
                num_boost_round = nrounds,
                evals = watchlist,
                verbose_eval = True
                )

[0]	eval-rmse:2.0056	train-rmse:2.00487
[1]	eval-rmse:1.90958	train-rmse:1.90889
[2]	eval-rmse:1.82076	train-rmse:1.8201
[3]	eval-rmse:1.7394	train-rmse:1.73862
[4]	eval-rmse:1.65689	train-rmse:1.65615
[5]	eval-rmse:1.57862	train-rmse:1.57782
[6]	eval-rmse:1.50444	train-rmse:1.50362
[7]	eval-rmse:1.43443	train-rmse:1.43351
[8]	eval-rmse:1.36794	train-rmse:1.36684
[9]	eval-rmse:1.3051	train-rmse:1.30383
[10]	eval-rmse:1.24578	train-rmse:1.24448
[11]	eval-rmse:1.19034	train-rmse:1.18892
[12]	eval-rmse:1.13689	train-rmse:1.1353
[13]	eval-rmse:1.08757	train-rmse:1.08583
[14]	eval-rmse:1.04003	train-rmse:1.03817
[15]	eval-rmse:0.995148	train-rmse:0.993058
[16]	eval-rmse:0.952726	train-rmse:0.950474
[17]	eval-rmse:0.912655	train-rmse:0.909898
[18]	eval-rmse:0.875215	train-rmse:0.872203
[19]	eval-rmse:0.839729	train-rmse:0.836364
[20]	eval-rmse:0.805995	train-rmse:0.80232
[21]	eval-rmse:0.775364	train-rmse:0.771327
[22]	eval-rmse:0.745339	train-rmse:0.740953
[23]	eval-rmse:0.717162	train-rmse

[187]	eval-rmse:0.333042	train-rmse:0.225253
[188]	eval-rmse:0.333021	train-rmse:0.224926
[189]	eval-rmse:0.333017	train-rmse:0.22458
[190]	eval-rmse:0.332987	train-rmse:0.224138
[191]	eval-rmse:0.332943	train-rmse:0.223944
[192]	eval-rmse:0.332934	train-rmse:0.223644
[193]	eval-rmse:0.332864	train-rmse:0.223346
[194]	eval-rmse:0.332825	train-rmse:0.222863
[195]	eval-rmse:0.332807	train-rmse:0.222794
[196]	eval-rmse:0.332802	train-rmse:0.222518
[197]	eval-rmse:0.332799	train-rmse:0.222398
[198]	eval-rmse:0.332766	train-rmse:0.222174
[199]	eval-rmse:0.332731	train-rmse:0.221978
[200]	eval-rmse:0.332712	train-rmse:0.22181
[201]	eval-rmse:0.332685	train-rmse:0.221644
[202]	eval-rmse:0.332673	train-rmse:0.221428
[203]	eval-rmse:0.332643	train-rmse:0.22116
[204]	eval-rmse:0.332638	train-rmse:0.221084
[205]	eval-rmse:0.33263	train-rmse:0.220994
[206]	eval-rmse:0.332612	train-rmse:0.220668
[207]	eval-rmse:0.332585	train-rmse:0.22035
[208]	eval-rmse:0.332579	train-rmse:0.220298
[209]	eval-rmse

[371]	eval-rmse:0.329566	train-rmse:0.190447
[372]	eval-rmse:0.329544	train-rmse:0.190186
[373]	eval-rmse:0.329529	train-rmse:0.190046
[374]	eval-rmse:0.329526	train-rmse:0.190002
[375]	eval-rmse:0.329518	train-rmse:0.189886
[376]	eval-rmse:0.329516	train-rmse:0.189749
[377]	eval-rmse:0.329494	train-rmse:0.189498
[378]	eval-rmse:0.329487	train-rmse:0.189285
[379]	eval-rmse:0.329489	train-rmse:0.189128
[380]	eval-rmse:0.329488	train-rmse:0.189094
[381]	eval-rmse:0.329484	train-rmse:0.18876
[382]	eval-rmse:0.32948	train-rmse:0.188634
[383]	eval-rmse:0.329488	train-rmse:0.188462
[384]	eval-rmse:0.329462	train-rmse:0.188255
[385]	eval-rmse:0.329456	train-rmse:0.188093
[386]	eval-rmse:0.329445	train-rmse:0.18791
[387]	eval-rmse:0.329445	train-rmse:0.187877
[388]	eval-rmse:0.329444	train-rmse:0.187848
[389]	eval-rmse:0.329446	train-rmse:0.187776
[390]	eval-rmse:0.329444	train-rmse:0.187741
[391]	eval-rmse:0.329413	train-rmse:0.18748
[392]	eval-rmse:0.32941	train-rmse:0.187416
[393]	eval-rmse

[554]	eval-rmse:0.328635	train-rmse:0.169749
[555]	eval-rmse:0.328632	train-rmse:0.169691
[556]	eval-rmse:0.328628	train-rmse:0.169606
[557]	eval-rmse:0.328635	train-rmse:0.16948
[558]	eval-rmse:0.32864	train-rmse:0.169399
[559]	eval-rmse:0.328638	train-rmse:0.169345
[560]	eval-rmse:0.328637	train-rmse:0.169236
[561]	eval-rmse:0.328636	train-rmse:0.16921
[562]	eval-rmse:0.328626	train-rmse:0.169108
[563]	eval-rmse:0.328622	train-rmse:0.16904
[564]	eval-rmse:0.328618	train-rmse:0.168937
[565]	eval-rmse:0.328614	train-rmse:0.168861
[566]	eval-rmse:0.328614	train-rmse:0.168779
[567]	eval-rmse:0.328604	train-rmse:0.168617
[568]	eval-rmse:0.328607	train-rmse:0.168483
[569]	eval-rmse:0.328606	train-rmse:0.168412
[570]	eval-rmse:0.328582	train-rmse:0.168125
[571]	eval-rmse:0.328582	train-rmse:0.16803
[572]	eval-rmse:0.328573	train-rmse:0.167965
[573]	eval-rmse:0.328571	train-rmse:0.167855
[574]	eval-rmse:0.328564	train-rmse:0.167688
[575]	eval-rmse:0.32855	train-rmse:0.167557
[576]	eval-rmse:

[738]	eval-rmse:0.328176	train-rmse:0.15448
[739]	eval-rmse:0.328173	train-rmse:0.154365
[740]	eval-rmse:0.328172	train-rmse:0.154348
[741]	eval-rmse:0.328172	train-rmse:0.154273
[742]	eval-rmse:0.328173	train-rmse:0.154207
[743]	eval-rmse:0.328171	train-rmse:0.154147
[744]	eval-rmse:0.328173	train-rmse:0.154044
[745]	eval-rmse:0.328177	train-rmse:0.153943
[746]	eval-rmse:0.32818	train-rmse:0.15391
[747]	eval-rmse:0.328178	train-rmse:0.153885
[748]	eval-rmse:0.328172	train-rmse:0.153796
[749]	eval-rmse:0.328164	train-rmse:0.153712
[750]	eval-rmse:0.328163	train-rmse:0.153571
[751]	eval-rmse:0.328166	train-rmse:0.153493
[752]	eval-rmse:0.328161	train-rmse:0.153389
[753]	eval-rmse:0.328162	train-rmse:0.153323
[754]	eval-rmse:0.328158	train-rmse:0.153278
[755]	eval-rmse:0.328158	train-rmse:0.153169
[756]	eval-rmse:0.328161	train-rmse:0.153074
[757]	eval-rmse:0.328157	train-rmse:0.153032
[758]	eval-rmse:0.328157	train-rmse:0.152926
[759]	eval-rmse:0.328155	train-rmse:0.152849
[760]	eval-rm

...

[1281]	eval-rmse:0.327565	train-rmse:0.12255
[1282]	eval-rmse:0.327566	train-rmse:0.122532
[1283]	eval-rmse:0.327566	train-rmse:0.122515
[1284]	eval-rmse:0.327565	train-rmse:0.122473
[1285]	eval-rmse:0.327566	train-rmse:0.12246
[1286]	eval-rmse:0.327566	train-rmse:0.122421
[1287]	eval-rmse:0.327561	train-rmse:0.122338
[1288]	eval-rmse:0.327561	train-rmse:0.122313
[1289]	eval-rmse:0.327557	train-rmse:0.122184
[1290]	eval-rmse:0.327558	train-rmse:0.122143
[1291]	eval-rmse:0.32756	train-rmse:0.122108
[1292]	eval-rmse:0.327561	train-rmse:0.122044
[1293]	eval-rmse:0.327559	train-rmse:0.121992
[1294]	eval-rmse:0.32756	train-rmse:0.121981
[1295]	eval-rmse:0.32756	train-rmse:0.121959
[1296]	eval-rmse:0.32756	train-rmse:0.121928
[1297]	eval-rmse:0.327559	train-rmse:0.121897
[1298]	eval-rmse:0.327561	train-rmse:0.121847
[1299]	eval-rmse:0.327561	train-rmse:0.121829
[1300]	eval-rmse:0.327561	train-rmse:0.121786
[1301]	eval-rmse:0.327556	train-rmse:0.121651
[1302]	eval-rmse:0.327557	train-rmse:0.1

[1460]	eval-rmse:0.327512	train-rmse:0.114567
[1461]	eval-rmse:0.327513	train-rmse:0.11451
[1462]	eval-rmse:0.327513	train-rmse:0.114463
[1463]	eval-rmse:0.327509	train-rmse:0.114358
[1464]	eval-rmse:0.327507	train-rmse:0.114299
[1465]	eval-rmse:0.327508	train-rmse:0.114223
[1466]	eval-rmse:0.327504	train-rmse:0.114155
[1467]	eval-rmse:0.327503	train-rmse:0.11408
[1468]	eval-rmse:0.327498	train-rmse:0.114014
[1469]	eval-rmse:0.327498	train-rmse:0.113998
[1470]	eval-rmse:0.327499	train-rmse:0.11394
[1471]	eval-rmse:0.327501	train-rmse:0.113876
[1472]	eval-rmse:0.3275	train-rmse:0.113839
[1473]	eval-rmse:0.3275	train-rmse:0.113776
[1474]	eval-rmse:0.3275	train-rmse:0.113752
[1475]	eval-rmse:0.327501	train-rmse:0.113731
[1476]	eval-rmse:0.327502	train-rmse:0.113701
[1477]	eval-rmse:0.327499	train-rmse:0.11362
[1478]	eval-rmse:0.327498	train-rmse:0.113606
[1479]	eval-rmse:0.327497	train-rmse:0.113569
[1480]	eval-rmse:0.327499	train-rmse:0.113497
[1481]	eval-rmse:0.327498	train-rmse:0.11344

[1639]	eval-rmse:0.327454	train-rmse:0.106982
[1640]	eval-rmse:0.327456	train-rmse:0.106961
[1641]	eval-rmse:0.327456	train-rmse:0.106929
[1642]	eval-rmse:0.327457	train-rmse:0.106894
[1643]	eval-rmse:0.327456	train-rmse:0.10685
[1644]	eval-rmse:0.327456	train-rmse:0.106834
[1645]	eval-rmse:0.327459	train-rmse:0.106787
[1646]	eval-rmse:0.32746	train-rmse:0.106755
[1647]	eval-rmse:0.327461	train-rmse:0.106732
[1648]	eval-rmse:0.327462	train-rmse:0.106697
[1649]	eval-rmse:0.327461	train-rmse:0.106677
[1650]	eval-rmse:0.32746	train-rmse:0.106659
[1651]	eval-rmse:0.327461	train-rmse:0.106639
[1652]	eval-rmse:0.327461	train-rmse:0.106572
[1653]	eval-rmse:0.327462	train-rmse:0.106532
[1654]	eval-rmse:0.327461	train-rmse:0.106505
[1655]	eval-rmse:0.327461	train-rmse:0.106444
[1656]	eval-rmse:0.327461	train-rmse:0.106433
[1657]	eval-rmse:0.327461	train-rmse:0.106373
[1658]	eval-rmse:0.327462	train-rmse:0.1063
[1659]	eval-rmse:0.327463	train-rmse:0.10626
[1660]	eval-rmse:0.327462	train-rmse:0.1

[1818]	eval-rmse:0.327425	train-rmse:0.100281
[1819]	eval-rmse:0.327425	train-rmse:0.100257
[1820]	eval-rmse:0.327425	train-rmse:0.100239
[1821]	eval-rmse:0.327424	train-rmse:0.100217
[1822]	eval-rmse:0.327425	train-rmse:0.100142
[1823]	eval-rmse:0.327425	train-rmse:0.100113
[1824]	eval-rmse:0.327426	train-rmse:0.100098
[1825]	eval-rmse:0.327426	train-rmse:0.100072
[1826]	eval-rmse:0.327426	train-rmse:0.100047
[1827]	eval-rmse:0.327426	train-rmse:0.100029
[1828]	eval-rmse:0.327425	train-rmse:0.099987
[1829]	eval-rmse:0.327425	train-rmse:0.099964
[1830]	eval-rmse:0.327424	train-rmse:0.099934
[1831]	eval-rmse:0.327425	train-rmse:0.099897
[1832]	eval-rmse:0.327425	train-rmse:0.099826
[1833]	eval-rmse:0.327424	train-rmse:0.099772
[1834]	eval-rmse:0.327426	train-rmse:0.099685
[1835]	eval-rmse:0.327426	train-rmse:0.099668
[1836]	eval-rmse:0.327426	train-rmse:0.099621
[1837]	eval-rmse:0.327426	train-rmse:0.099581
[1838]	eval-rmse:0.327426	train-rmse:0.09957
[1839]	eval-rmse:0.327424	train-rms

[1997]	eval-rmse:0.327426	train-rmse:0.094502
[1998]	eval-rmse:0.327426	train-rmse:0.094468
[1999]	eval-rmse:0.327426	train-rmse:0.094428


In [43]:
#Test predictions
pred = np.exp(gbm.predict(xgb.DMatrix(X_test))) - 1

In [44]:
#Use mean absolute error to get a basic estimate of the error
mae = (abs(pred - y_test)).mean()
mae

4.827394962310791

In [49]:
#Take a look at feature importance
feature_scores = gbm.get_fscore()
feature_scores

{'dropoff_latitude': 610689,
 'dropoff_longitude': 622853,
 'latitude_difference': 498608,
 'longitude_difference': 438287,
 'passenger_count': 227133,
 'pickup_day': 387770,
 'pickup_hour': 323986,
 'pickup_latitude': 753138,
 'pickup_longitude': 829288,
 'pickup_minute': 413378,
 'pickup_month': 227789,
 'pickup_weekday': 216596,
 'store_and_fwd_flag': 5359,
 'trip_distance': 453975}

In [50]:
#This is not very telling, so let's scale the features
summ = 0
for key in feature_scores:
    summ = summ + feature_scores[key]

for key in feature_scores:
    feature_scores[key] = feature_scores[key] / summ

feature_scores

{'dropoff_latitude': 0.10163161031338946,
 'dropoff_longitude': 0.10365595807117137,
 'latitude_difference': 0.08297895320717828,
 'longitude_difference': 0.07294025860859542,
 'passenger_count': 0.037799751666250894,
 'pickup_day': 0.06453315768127972,
 'pickup_hour': 0.05391814638710342,
 'pickup_latitude': 0.12533814712268523,
 'pickup_longitude': 0.13801112326170953,
 'pickup_minute': 0.06879487236241084,
 'pickup_month': 0.03790892398860414,
 'pickup_weekday': 0.03604617123845182,
 'store_and_fwd_flag': 0.0008918513345900355,
 'trip_distance': 0.07555107475657984}

# Save the model

In [48]:
filename = "xgb_model.sav"
pickle.dump(gbm, open(filename, 'wb'))